In [1]:
import numpy as np
import deep_laa_support as dls
import random
import sys
import tensorflow as tf
from sklearn.cluster import KMeans

# convert data from .mat to .npz
# in_file = "feature_31_L"
# dls.convert_mat_to_one_hot_npz(in_file)
# print('file ' + in_file + '.npz ' 'saved.')

# read data
filename = 'feature_rotten_tomatoes'
# filename = 'feature_31_L'
data_all = np.load(filename +'.npz')
print('File ' + filename + '.npz ' 'loaded.')
user_labels = data_all['user_labels']
true_labels = data_all['true_labels']
category_size = data_all['category_num']
source_num = data_all['source_num']
feature = data_all['feature']
_, feature_size = np.shape(feature)
n_samples, _ = np.shape(true_labels)

#================= basic parameters =====================
# define batch size (use all samples in one batch)
batch_size = n_samples # n_samples
cluster_num = 200
T = 1 # mc_sampling_times

if np.max(feature) <= 1 and np.min(feature) >= 0:
    flag_node_type = 'Bernoulli'
else:
    flag_node_type = 'Gaussian'   
print('Using ' + flag_node_type + ' output nodes.')

#================= encoder q(y|l) and q(h|x) =====================
with tf.name_scope('encoder'):
    #================= q(y|l) =====================
    # define input l (source label vectors)
    input_size = source_num * category_size
    with tf.variable_scope('q_yl'):
        l = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_size], name='l_input')
        pi_yl, weights_yl, biases_yl = dls.LAA_encoder(l, batch_size, source_num, category_size)
    # loss: cross entropy between y_classifier and y_target for pre-training classifier
    with tf.variable_scope('q_yl'):
        pi_yl_target = tf.placeholder(dtype=tf.float32, shape=[batch_size, category_size], name='pi_yl_target')
        loss_yl = dls.LAA_loss_classifier(pi_yl, pi_yl_target)
    # optimizier
    learning_rate_yl = 0.01
    optimizer_pre_train_yl = tf.train.AdamOptimizer(learning_rate=learning_rate_yl).minimize(loss_yl)

    #================= q(h|x) =====================
    h1_size_encoder = int(np.floor(feature_size/2.0))
    h2_size_encoder = 100 # int(np.floor(feature_size/4.0))
    embedding_size = 50 # int(np.floor(feature_size/8.0))
    h1_size_decoder = 100 # int(np.floor(feature_size/4.0))
    h2_size_decoder = int(np.floor(feature_size/2.0))
    # h1_size_encoder = int(np.floor(feature_size/2.0))
    # h2_size_encoder = int(np.floor(feature_size/4.0))
    # embedding_size = int(np.floor(feature_size/8.0))
    # h1_size_decoder = int(np.floor(feature_size/4.0))
    # h2_size_decoder = int(np.floor(feature_size/2.0))
    with tf.variable_scope('q_hx'):
        x = tf.placeholder(dtype=tf.float32, shape=[batch_size, feature_size], name='x_input')
        # mu_hx[batch_size, embedding_size]
        # sigma_hx[batch_size, embedding_size]
        with tf.variable_scope('feature_encoder_h1'):
            _h1_encoder, w1_encoder, b1_encoder = dls.full_connect_relu_BN(x, [feature_size, h1_size_encoder])
        with tf.variable_scope('feature_encoder_h2'):
            _h2_encoder, w2_encoder, b2_encoder = dls.full_connect_relu_BN(_h1_encoder, [h1_size_encoder, h2_size_encoder])
        with tf.variable_scope('feature_encoder_mu'):
            mu_hx, w_mu_encoder, b_mu_encoder = dls.full_connect(_h2_encoder, [h2_size_encoder, embedding_size])
        with tf.variable_scope('feature_encoder_sigma'):
            sigma_hx, w_sigma_encoder, b_sigma_encoder = dls.full_connect(_h2_encoder, [h2_size_encoder, embedding_size])
        # mu_hx, sigma_hx = dls.vae_encoder(x, feature_size, h1_size_encoder, h2_size_encoder, embedding_size)
        # embedding_h[batch_size, T, embedding_size]
        embedding_h = tf.reshape(mu_hx, [batch_size, 1, embedding_size]) \
            + tf.reshape(sigma_hx, [batch_size, 1, -1]) \
            * tf.random_normal(shape=[batch_size, T, embedding_size], mean=0, stddev=1, dtype=tf.float32)

    with tf.variable_scope('q_hx_AE'):
        # x_reconstr, _, _ = dls.vae_decoder(mu_hx, embedding_size, h1_size_decoder, h2_size_decoder, feature_size)
        with tf.variable_scope('feature_decoder_h1'):
            _h1_decoder, w1_decoder, b1_decoder = dls.full_connect_relu_BN(mu_hx, [embedding_size, h1_size_decoder])
        with tf.variable_scope('feature_decoder_h2'):
            _h2_decoder, w2_decoder, b2_decoder = dls.full_connect_relu_BN(_h1_decoder, [h1_size_decoder, h2_size_decoder])
        with tf.variable_scope('feature_decoder_rho'):
            if flag_node_type == 'Bernoulli':
                x_reconstr, w_rho_decoder, b_rho_decoder = dls.full_connect_sigmoid(_h2_decoder, [h2_size_decoder, feature_size])
            elif flag_node_type == 'Gaussian':
                x_reconstr, w_rho_decoder, b_rho_decoder = dls.full_connect(_h2_decoder, [h2_size_decoder, feature_size])
        
        # Bernoulli
        loss_cross_entropy_AE = -tf.reduce_mean(tf.reduce_sum(x*tf.log(1e-10+x_reconstr) + (1.0-x)*tf.log(1e-10+(1.0-x_reconstr)), -1))
        # Gaussian
        loss_square_AE = 0.5 * tf.reduce_mean(tf.square(x_reconstr - x))
        constraint_w_AE = 0.5 * (tf.reduce_mean(tf.square(w1_encoder)) + tf.reduce_mean(tf.square(b1_encoder))
            + tf.reduce_mean(tf.square(w2_encoder)) + tf.reduce_mean(tf.square(b2_encoder))
            + tf.reduce_mean(tf.square(w_mu_encoder)) + tf.reduce_mean(tf.square(b_mu_encoder))
            + tf.reduce_mean(tf.square(w1_decoder)) + tf.reduce_mean(tf.square(b1_decoder))
            + tf.reduce_mean(tf.square(w2_decoder)) + tf.reduce_mean(tf.square(b2_decoder))
            + tf.reduce_mean(tf.square(w_rho_decoder)) + tf.reduce_mean(tf.square(b_rho_decoder)))
        if flag_node_type == 'Bernoulli':
            loss_AE = loss_cross_entropy_AE \
                + constraint_w_AE
        elif flag_node_type == 'Gaussian':
            loss_AE = loss_square_AE \
                + constraint_w_AE
        learning_rate_AE = 0.02
        optimizer_AE = tf.train.AdamOptimizer(learning_rate=learning_rate_AE).minimize(loss_AE)

    #================= p(x|h) =====================
    with tf.variable_scope('q_hx_AE'):
        with tf.variable_scope('feature_decoder_h1', reuse=True):
            _h_VAE = tf.reshape(embedding_h, [-1, embedding_size])
            _h1_decoder_VAE, _, _ = dls.full_connect_relu_BN(_h_VAE, [embedding_size, h1_size_decoder])
        with tf.variable_scope('feature_decoder_h2', reuse=True):
            _h2_decoder_VAE, _, _ = dls.full_connect_relu_BN(_h1_decoder_VAE, [h1_size_decoder, h2_size_decoder])
        with tf.variable_scope('feature_decoder_rho', reuse=True):
            if flag_node_type == 'Bernoulli':
                mu_xh, _, _ = dls.full_connect_sigmoid(_h2_decoder_VAE, [h2_size_decoder, feature_size])
            elif flag_node_type == 'Gaussian':
                mu_xh, _, _ = dls.full_connect(_h2_decoder_VAE, [h2_size_decoder, feature_size])
            mu_xh = tf.reshape(mu_xh, [batch_size, T, feature_size])
            
    print('Encoders are constructed.')
    
#================= decoder p(l|y), p(x|h), p(y|z), p(h|z) and p(z) =====================
with tf.name_scope('decoder'):
    #================= p(l|y) =====================
    with tf.variable_scope('p_ly'):
        # pi_ly[category_size, 1, source_num*category_size]
        pi_ly, weights_ly, biases_ly = dls.LAA_decoder(source_num, category_size)

        constraint_w_LAA = 0.5 * (tf.reduce_mean(tf.square(weights_ly)) + tf.reduce_mean(tf.square(biases_ly))
            + tf.reduce_mean(tf.square(weights_yl)) + tf.reduce_mean(tf.square(biases_yl)))
        
    #================= p(y|z) =====================
    with tf.variable_scope('p_yz'):
        # pi_yz[cluster_num, category_size]
        _pi_yz = tf.get_variable('pi_yz', dtype=tf.float32, 
                                initializer=tf.random_normal(shape=[cluster_num, category_size], mean=0, stddev=1, dtype=tf.float32))
        __pi_yz = tf.exp(_pi_yz)
        pi_yz = tf.div(__pi_yz, tf.reduce_sum(__pi_yz, -1, keepdims=True))
        
        pi_yz_assign = tf.placeholder(dtype=tf.float32, shape=[cluster_num, category_size], name='pi_yz_assign')
        initialize_pi_yz = tf.assign(_pi_yz, pi_yz_assign)
        
    #================= p(h|z) =====================
    with tf.variable_scope('p_hz'):
        # mu_hz[cluster_num, embedding_size]
        # sigma_hz[cluster_num, embedding_size]
        mu_hz = tf.get_variable('mu_hz', dtype=tf.float32, initializer=tf.random_normal(shape=[cluster_num, embedding_size], mean=0, stddev=1, dtype=tf.float32))
        sigma_hz = tf.get_variable('sigma_hz', dtype=tf.float32, initializer=tf.ones([cluster_num, embedding_size], dtype=tf.float32))

        mu_hz_assign = tf.placeholder(dtype=tf.float32, shape=[cluster_num, embedding_size], name='mu_hz_assign')
        initialize_mu_hz = tf.assign(mu_hz, mu_hz_assign)
        
    #================= p(z) =====================
    with tf.variable_scope('p_z'):
        # pi_z_prior[batch_size, cluster_num]
        pi_z_prior = tf.placeholder(dtype=tf.float32, shape=[batch_size, cluster_num], name='pi_z_prior')
        _pi_z = tf.get_variable('pi_z', dtype=tf.float32, initializer=tf.ones([batch_size, cluster_num]))
        __pi_z = tf.exp(_pi_z)
        pi_z = tf.div(__pi_z, tf.reduce_sum(__pi_z, -1, keepdims=True))

        pi_z_assign = tf.placeholder(dtype=tf.float32, shape=[batch_size, cluster_num], name='pi_z_assign')
        initialize_pi_z = tf.assign(_pi_z, pi_z_assign)
    print('Decoders are constructed.')
    
#================= elbo =====================
'''
q(h|x) log p(x|h)
q(y|l) log p(l|y)
q(h|x) log q(h|x)
q(y|l) log q(y|l)
q(z|x,l)q(h|x) log p(h|z)
q(z|x,l)q(y|l) log p(y|z)
q(z|x,l) log p(z)
q(z|x,l) log q(z|x,l)
q(z|x,l)
'''
with tf.name_scope('elbo'):
    #================= q(h|x) log p(x|h) =====================
    with tf.name_scope('q_hx_log_p_xh'):
        # reduce_mean along both T and batch_size
        _tmp = tf.reshape(x, [batch_size, 1, feature_size])
        if flag_node_type == 'Bernoulli':
            elbo_q_hx_log_p_xh = tf.reduce_mean(tf.reduce_sum(_tmp*tf.log(1e-10+mu_xh) + (1.0-_tmp)*tf.log(1e-10+(1.0-mu_xh)), -1))
        elif flag_node_type == 'Gaussian':
            elbo_q_hx_log_p_xh = -0.5 * tf.reduce_mean(tf.reduce_sum(tf.square(_tmp-mu_xh), -1))
    
    #================= q(y|l) log p(l|y) =====================
    with tf.name_scope('q_yl_log_p_ly'):
        elbo_q_yl_log_p_ly = -dls.LAA_loss_reconstr(l, pi_ly, pi_yl)
        
    #================= q(h|x) log q(h|x) =====================
    with tf.name_scope('q_hx_log_q_hx'):
        elbo_q_hx_log_q_hx = -0.5 * tf.reduce_mean(tf.reduce_sum(tf.log(1e-10+tf.square(sigma_hx)), -1))

    #================= q(y|l) log q(y|l) =====================
    with tf.name_scope('q_yl_log_q_yl'):
        elbo_q_yl_log_q_yl = tf.reduce_mean(tf.reduce_sum(pi_yl * tf.log(1e-10+pi_yl), -1))
    
    #================= q(z|x,l) =====================
    with tf.name_scope('q_zxl'):
        # p(h|z)[batch_size, T, cluster_num, 1]
        _h = tf.reshape(embedding_h, [batch_size, T, 1, embedding_size])
        _p_hz = -0.5 * tf.reduce_sum(
            tf.div(tf.square(_h-mu_hz), 1e-10+tf.square(sigma_hz)) 
            + tf.log(1e-10 + tf.square(sigma_hz)), -1, keepdims=True)
        # p_zhy[batch_size, T, cluster_num, category_size]
        _p_zhy = tf.log(1e-10+pi_yz) + _p_hz + tf.log(1e-10+tf.reshape(pi_z, [batch_size, 1, cluster_num, 1]))
        _p_zhy_max = tf.reduce_max(_p_zhy, 2, keepdims=True)
        p_zhy = tf.exp(_p_zhy - (_p_zhy_max + tf.log(1e-10+tf.reduce_sum(tf.exp(_p_zhy-_p_zhy_max), 2, keepdims=True))))
        
        # q_zxl[batch_size, cluster_num]
        # reduce_mean along both category_size and T
        _q_zxl = tf.reduce_sum(tf.reshape(pi_yl, [batch_size, 1, 1, category_size]) * p_zhy, -1)
        q_zxl = tf.reduce_mean(_q_zxl, 1)
        
        # z_index[batch_size]
        z_index = tf.argmax(q_zxl, 1)
        # cluster_pi_max[batch_size, category_size]
        # cluster_pi_avg[batch_size, category_size]
        cluster_pi_max = tf.gather(pi_yz, z_index)
        cluster_pi_avg = tf.matmul(q_zxl, pi_yz)
        
    #================= q(z|x,l)q(h|x) log p(h|z) =====================
    #================= q(h|x) log p(h|z) [batch_size, cluster_num] =====================
    with tf.name_scope('q_zxl_q_hx_log_p_hz'):
        # mu_hx[batch_size, embedding_size]
        # sigma_hx[batch_size, embedding_size]
        # mu_hz[cluster_num, embedding_size]
        # sigma_hz[cluster_num, embedding_size]
        _part_1 = tf.div(tf.square(tf.reshape(mu_hx, [batch_size, 1, embedding_size]) - mu_hz), 1e-10+tf.square(sigma_hz))
        _part_2 = tf.div(tf.square(tf.reshape(sigma_hx, [batch_size, 1, -1])), 1e-10+tf.square(sigma_hz))
        _part_3 = tf.log(1e-10 + tf.square(sigma_hz))
        # elbo_q_hx_log_p_hz[batch_size, cluster_num]
        elbo_q_hx_log_p_hz = -0.5 * tf.reduce_sum(_part_1 + _part_2 + _part_3, -1)
        elbo_q_zxl_q_hx_log_p_hz = tf.reduce_mean(tf.reduce_sum(q_zxl * elbo_q_hx_log_p_hz, -1))
    
    #================= q(z|x,l)q(y|l) log p(y|z) =====================
    #================= q(y|l) log p(y|z) [batch_size, cluster_num] =====================
    with tf.name_scope('q_zxl_q_yl_log_p_yz'):
        # pi_yz[cluster_num, category_size]
        # pi_yl[batch_size, category_size]
        # elbo_q_yl_log_p_yz[batch_size, cluster_num]
        elbo_q_yl_log_p_yz = tf.reduce_sum(tf.reshape(pi_yl, [batch_size, 1, category_size]) * tf.log(1e-10 + pi_yz), -1)
        elbo_q_zxl_q_yl_log_p_yz = tf.reduce_mean(tf.reduce_sum(q_zxl * elbo_q_yl_log_p_yz, -1))
    
    #================= q(z|x,l) log p(z) =====================
    #================= log p(z) [cluster_num] =====================
    with tf.name_scope('q_zxl_log_p_z'):
        # elbo_log_p_z[batch_size, cluster_num]
        elbo_log_p_z = tf.log(1e-10 + pi_z)
        elbo_q_zxl_log_p_z = tf.reduce_mean(tf.reduce_sum(q_zxl * elbo_log_p_z, -1))
    
    #================= q(z|x,l) log q(z|x,l) =====================
    with tf.name_scope('q_zxl_log_q_zxl'):
        # q_zxl[batch_size, cluster_num]
        elbo_q_zxl_log_q_zxl = tf.reduce_mean(tf.reduce_sum(q_zxl * tf.log(1e-10 + q_zxl), -1))
    
    #================= overall elbo =====================
    elbo = elbo_q_hx_log_p_xh + elbo_q_yl_log_p_ly - elbo_q_hx_log_q_hx - elbo_q_yl_log_q_yl \
        + elbo_q_zxl_q_hx_log_p_hz + elbo_q_zxl_q_yl_log_p_yz + elbo_q_zxl_log_p_z - elbo_q_zxl_log_q_zxl    
    
    q_zxl_entropy = -elbo_q_zxl_log_q_zxl
    
    with tf.variable_scope('regularization_prior'):
        mu_hz_prior_mu = tf.placeholder(dtype=tf.float32, shape=[cluster_num, embedding_size], name='mu_hz_prior_mu')
        # sigma_hz_prior_alpha = tf.placeholder(dtype=tf.float32, shape=[cluster_num, embedding_size], name='sigma_hz_prior')
        pi_yz_prior = tf.placeholder(dtype=tf.float32, shape=[cluster_num, category_size], name='pi_yz_prior')
    
        constraint_prior = 0.5*tf.reduce_mean(tf.square(mu_hz - mu_hz_prior_mu)) \
            - tf.reduce_mean(pi_yz_prior * tf.log(1e-10+pi_yz)) \
            - tf.reduce_mean(pi_z_prior * tf.log(1e-10+pi_z)) \
            + tf.reduce_mean(1.0*tf.log(1e-10+tf.square(sigma_hz))+tf.div(2.0, 1e-10+tf.square(sigma_hz)))
    
    loss_overall = -elbo \
        + constraint_w_AE \
        + constraint_w_LAA \
        + 1.0 * constraint_prior
    
    # optimizier
    learning_rate_overall = 0.001
    optimizer_overall = tf.train.AdamOptimizer(learning_rate=learning_rate_overall).minimize(loss_overall)

    print('Clustering-based label-aware autoencoder is constructed.')

saver = tf.train.Saver()

File feature_rotten_tomatoes.npz loaded.
Using Gaussian output nodes.
Encoders are constructed.
Decoders are constructed.
Clustering-based label-aware autoencoder is constructed.


In [2]:
#================= training and inference =====================
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #================= pre-train pi_yl =====================
    # assign batch variables (use whole data in one batch)
    # define majority voting regularizer
    majority_y = dls.get_majority_y(user_labels, source_num, category_size)
    # pre-train classifier
    print("Pre-train pi_yl ...")
    epochs = 50
    for epoch in range(epochs):
        _, monitor_pi_yl = sess.run([optimizer_pre_train_yl, pi_yl], 
            feed_dict={l:user_labels, pi_yl_target:majority_y})
        if epoch % 10 == 0:
            hit_num = dls.cal_hit_num(true_labels, monitor_pi_yl)
            print("epoch: {0} accuracy: {1}".format(epoch, float(hit_num)/n_samples))
    
    print("Pre-train hx_AE ...")
    epochs = 2000
    for epoch in range(epochs):
        _, monitor_loss_square_AE, monitor_mu_hx = sess.run([optimizer_AE, loss_square_AE, mu_hx], 
            feed_dict={x:feature})
        if epoch % 50 == 0:
            print("epoch: {0} loss: {1}".format(epoch, monitor_loss_square_AE))    
    
    #================= calculate initial parameters =====================
    clustering_result = KMeans(n_clusters=cluster_num).fit(np.concatenate((monitor_mu_hx, majority_y), 1))
    # pi_z_prior_cluster = np.ones([n_samples, cluster_num]) / cluster_num
    pi_z_prior_cluster = dls.convert_to_one_hot(clustering_result.labels_, cluster_num, smooth=0.2)
    _ = sess.run(initialize_mu_hz, {mu_hz_assign:clustering_result.cluster_centers_[:, 0:embedding_size]})
    # pi_yz_prior_cluster = np.ones([cluster_num, category_size]) / cluster_num
    pi_yz_prior_cluster = dls.get_cluster_majority_y(
        clustering_result.labels_, user_labels, cluster_num, source_num, category_size)
    _ = sess.run(initialize_pi_yz, {pi_yz_assign:pi_yz_prior_cluster})
    _ = sess.run(initialize_pi_z, {pi_z_assign:pi_z_prior_cluster})
    
    mu_hz_prior_mu_cluster = clustering_result.cluster_centers_[:, 0:embedding_size]
    
    predict_label = np.zeros([batch_size, category_size])
    for i in range(batch_size):
        predict_label[i] = pi_yz_prior_cluster[clustering_result.labels_[i], :]
    print("Initial clustering accuracy: {0}".format(float(dls.cal_hit_num(true_labels, predict_label)) / n_samples))
    
    #================= save current model =====================
    saved_path = saver.save(sess, './my_model')

Pre-train pi_yl ...
epoch: 0 accuracy: 0.6949389877975595
epoch: 10 accuracy: 0.8825765153030606
epoch: 20 accuracy: 0.8857771554310863
epoch: 30 accuracy: 0.8821764352870574
epoch: 40 accuracy: 0.8789757951590318
Pre-train hx_AE ...
epoch: 0 loss: 0.0034235448110848665
epoch: 50 loss: 0.003666071454063058
epoch: 100 loss: 0.00311955320648849
epoch: 150 loss: 0.0031102949287742376
epoch: 200 loss: 0.003106012474745512
epoch: 250 loss: 0.0031006676144897938
epoch: 300 loss: 0.0030889601912349463
epoch: 350 loss: 0.003067582380026579
epoch: 400 loss: 0.0030528968200087547
epoch: 450 loss: 0.0030483026057481766
epoch: 500 loss: 0.0030430324841290712
epoch: 550 loss: 0.0030466991011053324
epoch: 600 loss: 0.003037713235244155
epoch: 650 loss: 0.0029859282076358795
epoch: 700 loss: 0.002978849457576871
epoch: 750 loss: 0.0029778298921883106
epoch: 800 loss: 0.0029789551626890898
epoch: 850 loss: 0.0029758764430880547
epoch: 900 loss: 0.0029715222772210836
epoch: 950 loss: 0.0029758405871689

In [3]:
with tf.Session() as sess:
    saver.restore(sess, './my_model')
    
    print("Train overall net ...")
    epochs = 2000
    for epoch in range(epochs):
        _, monitor_loss_overall, monitor_pi_yl, monitor_cluster_pi_max, monitor_cluster_pi_avg, \
            monitor_constraint_w_AE, monitor_constraint_prior = sess.run(
                [optimizer_overall, loss_overall, pi_yl, cluster_pi_max, cluster_pi_avg, constraint_w_AE, constraint_prior], 
                feed_dict={l:user_labels, x:feature, 
                           pi_z_prior:pi_z_prior_cluster, 
                           mu_hz_prior_mu:mu_hz_prior_mu_cluster, 
                           pi_yz_prior:pi_yz_prior_cluster})
        if epoch % 10 == 0:
            print("epoch: {0} loss: {1}".format(epoch, monitor_loss_overall))
            print("epoch: {0} loss: {1}".format(epoch, monitor_constraint_w_AE))
            print("epoch: {0} loss: {1}".format(epoch, monitor_constraint_prior))
            hit_num_object_level = dls.cal_hit_num(true_labels, monitor_pi_yl)
            hit_num_cluster_level_max = dls.cal_hit_num(true_labels, monitor_cluster_pi_max)
            hit_num_cluster_level_avg = dls.cal_hit_num(true_labels, monitor_cluster_pi_avg)
            print("epoch: {0} accuracy(object level): {1}".format(epoch, float(hit_num_object_level)/n_samples))
            print("epoch: {0} accuracy(cluster level max): {1}".format(epoch, float(hit_num_cluster_level_max)/n_samples))
            print("epoch: {0} accuracy(cluster level avg): {1}".format(epoch, float(hit_num_cluster_level_avg)/n_samples))
    print("Training overall net. Done!")
    
    monitor_q_zxl = sess.run(q_zxl, 
                feed_dict={l:user_labels, x:feature, 
                           pi_z_prior:pi_z_prior_cluster, 
                           mu_hz_prior_mu:mu_hz_prior_mu_cluster, 
                           pi_yz_prior:pi_yz_prior_cluster})

INFO:tensorflow:Restoring parameters from ./my_model
Train overall net ...
epoch: 0 loss: 212.71759033203125
epoch: 0 loss: 0.0003140648186672479
epoch: 0 loss: 2.2800440788269043
epoch: 0 accuracy(object level): 0.8791758351670335
epoch: 0 accuracy(cluster level max): 0.8849769953990798
epoch: 0 accuracy(cluster level avg): 0.8797759551910382
epoch: 10 loss: 119.65145874023438
epoch: 10 loss: 0.00034990295534953475
epoch: 10 loss: 2.2991762161254883
epoch: 10 accuracy(object level): 0.8789757951590318
epoch: 10 accuracy(cluster level max): 0.8811762352470494
epoch: 10 accuracy(cluster level avg): 0.8701740348069614
epoch: 20 loss: 90.2087631225586
epoch: 20 loss: 0.0003818291297648102
epoch: 20 loss: 2.319314479827881
epoch: 20 accuracy(object level): 0.8779755951190238
epoch: 20 accuracy(cluster level max): 0.8799759951990398
epoch: 20 accuracy(cluster level avg): 0.8569713942788558
epoch: 30 loss: 74.88299560546875
epoch: 30 loss: 0.0004009485128335655
epoch: 30 loss: 2.339801788330

epoch: 300 loss: 38.76754379272461
epoch: 300 loss: 0.0005927612655796111
epoch: 300 loss: 2.528430700302124
epoch: 300 accuracy(object level): 0.8975795159031806
epoch: 300 accuracy(cluster level max): 0.8609721944388877
epoch: 300 accuracy(cluster level avg): 0.8825765153030606
epoch: 310 loss: 38.73283004760742
epoch: 310 loss: 0.0006033835816197097
epoch: 310 loss: 2.5267350673675537
epoch: 310 accuracy(object level): 0.8981796359271854
epoch: 310 accuracy(cluster level max): 0.8583716743348669
epoch: 310 accuracy(cluster level avg): 0.8821764352870574
epoch: 320 loss: 38.48049545288086
epoch: 320 loss: 0.0006138171884231269
epoch: 320 loss: 2.525012731552124
epoch: 320 accuracy(object level): 0.8983796759351871
epoch: 320 accuracy(cluster level max): 0.8589717943588717
epoch: 320 accuracy(cluster level avg): 0.8831766353270654
epoch: 330 loss: 38.18124771118164
epoch: 330 loss: 0.0006196586764417589
epoch: 330 loss: 2.5225632190704346
epoch: 330 accuracy(object level): 0.898579715

epoch: 600 loss: 35.76594543457031
epoch: 600 loss: 0.0005749156698584557
epoch: 600 loss: 2.405994176864624
epoch: 600 accuracy(object level): 0.9013802760552111
epoch: 600 accuracy(cluster level max): 0.8829765953190638
epoch: 600 accuracy(cluster level avg): 0.8973794758951791
epoch: 610 loss: 35.71345901489258
epoch: 610 loss: 0.0005732803838327527
epoch: 610 loss: 2.401853084564209
epoch: 610 accuracy(object level): 0.9019803960792159
epoch: 610 accuracy(cluster level max): 0.8831766353270654
epoch: 610 accuracy(cluster level avg): 0.8979795959191839
epoch: 620 loss: 35.66580581665039
epoch: 620 loss: 0.0005715704755857587
epoch: 620 loss: 2.3977441787719727
epoch: 620 accuracy(object level): 0.9019803960792159
epoch: 620 accuracy(cluster level max): 0.882376475295059
epoch: 620 accuracy(cluster level avg): 0.8987797559511902
epoch: 630 loss: 35.617027282714844
epoch: 630 loss: 0.0005700418259948492
epoch: 630 loss: 2.3937206268310547
epoch: 630 accuracy(object level): 0.902380476

epoch: 900 loss: 34.74263000488281
epoch: 900 loss: 0.0005633828113786876
epoch: 900 loss: 2.294365406036377
epoch: 900 accuracy(object level): 0.9047809561912382
epoch: 900 accuracy(cluster level max): 0.8927785557111422
epoch: 900 accuracy(cluster level avg): 0.9041808361672334
epoch: 910 loss: 34.72320556640625
epoch: 910 loss: 0.0005644999910145998
epoch: 910 loss: 2.2910397052764893
epoch: 910 accuracy(object level): 0.9047809561912382
epoch: 910 accuracy(cluster level max): 0.8923784756951391
epoch: 910 accuracy(cluster level avg): 0.9041808361672334
epoch: 920 loss: 34.70207977294922
epoch: 920 loss: 0.0005645641940645874
epoch: 920 loss: 2.2877237796783447
epoch: 920 accuracy(object level): 0.9047809561912382
epoch: 920 accuracy(cluster level max): 0.8933786757351471
epoch: 920 accuracy(cluster level avg): 0.9047809561912382
epoch: 930 loss: 34.679054260253906
epoch: 930 loss: 0.0005648487131111324
epoch: 930 loss: 2.2843971252441406
epoch: 930 accuracy(object level): 0.9045809

epoch: 1190 loss: 34.20328140258789
epoch: 1190 loss: 0.0005744292866438627
epoch: 1190 loss: 2.2060000896453857
epoch: 1190 accuracy(object level): 0.9055811162232447
epoch: 1190 accuracy(cluster level max): 0.8979795959191839
epoch: 1190 accuracy(cluster level avg): 0.9051810362072414
epoch: 1200 loss: 34.18685531616211
epoch: 1200 loss: 0.0005748241674154997
epoch: 1200 loss: 2.2032387256622314
epoch: 1200 accuracy(object level): 0.9055811162232447
epoch: 1200 accuracy(cluster level max): 0.8977795559111822
epoch: 1200 accuracy(cluster level avg): 0.905381076215243
epoch: 1210 loss: 34.18338394165039
epoch: 1210 loss: 0.000575603567995131
epoch: 1210 loss: 2.2005107402801514
epoch: 1210 accuracy(object level): 0.9055811162232447
epoch: 1210 accuracy(cluster level max): 0.8981796359271854
epoch: 1210 accuracy(cluster level avg): 0.9047809561912382
epoch: 1220 loss: 34.17121887207031
epoch: 1220 loss: 0.0005754667799919844
epoch: 1220 loss: 2.197814464569092
epoch: 1220 accuracy(objec

epoch: 1480 loss: 33.8864631652832
epoch: 1480 loss: 0.000580094289034605
epoch: 1480 loss: 2.1351675987243652
epoch: 1480 accuracy(object level): 0.9065813162632527
epoch: 1480 accuracy(cluster level max): 0.9007801560312062
epoch: 1480 accuracy(cluster level avg): 0.9059811962392479
epoch: 1490 loss: 33.87968444824219
epoch: 1490 loss: 0.000579789571929723
epoch: 1490 loss: 2.1330301761627197
epoch: 1490 accuracy(object level): 0.9065813162632527
epoch: 1490 accuracy(cluster level max): 0.9011802360472094
epoch: 1490 accuracy(cluster level avg): 0.9061812362472494
epoch: 1500 loss: 33.863197326660156
epoch: 1500 loss: 0.0005800488870590925
epoch: 1500 loss: 2.1309001445770264
epoch: 1500 accuracy(object level): 0.9067813562712542
epoch: 1500 accuracy(cluster level max): 0.9009801960392079
epoch: 1500 accuracy(cluster level avg): 0.9061812362472494
epoch: 1510 loss: 33.85419464111328
epoch: 1510 loss: 0.0005802249652333558
epoch: 1510 loss: 2.1287825107574463
epoch: 1510 accuracy(obje

epoch: 1770 loss: 33.67511749267578
epoch: 1770 loss: 0.0005837493226863444
epoch: 1770 loss: 2.0805511474609375
epoch: 1770 accuracy(object level): 0.906381276255251
epoch: 1770 accuracy(cluster level max): 0.9021804360872174
epoch: 1770 accuracy(cluster level avg): 0.9069813962792559
epoch: 1780 loss: 33.669593811035156
epoch: 1780 loss: 0.000583887449465692
epoch: 1780 loss: 2.0789573192596436
epoch: 1780 accuracy(object level): 0.906381276255251
epoch: 1780 accuracy(cluster level max): 0.9017803560712142
epoch: 1780 accuracy(cluster level avg): 0.9069813962792559
epoch: 1790 loss: 33.65970230102539
epoch: 1790 loss: 0.0005842767423018813
epoch: 1790 loss: 2.0773561000823975
epoch: 1790 accuracy(object level): 0.906381276255251
epoch: 1790 accuracy(cluster level max): 0.9023804760952191
epoch: 1790 accuracy(cluster level avg): 0.9067813562712542
epoch: 1800 loss: 33.662322998046875
epoch: 1800 loss: 0.0005850078305229545
epoch: 1800 loss: 2.0757927894592285
epoch: 1800 accuracy(obje

In [4]:
import scipy.io
if False:
    scipy.io.savemat('trial_tf_workplace', 
                     {'monitor_q_zxl':monitor_q_zxl})